In [1]:
%matplotlib inline

import numpy as np
import pyvista as pv
from scipy.spatial import KDTree
from binary_to_csv import binary_to_csv
import os
import pandas as pd

In [24]:
DATA_DIR = '../data/results'

csvs = [f for f in os.listdir(DATA_DIR) if f.endswith('.csv')]
csvs.sort()

dataframes = {}
for filename in csvs:
    dataframes[filename] = pd.read_csv(os.path.join(DATA_DIR, filename), header=None)


for [key, value] in dataframes.items():
   print(f"{key}: {len(value)} rows")


1-data_20241115_120259_pre.csv: 162 rows
1-data_20241115_120617_post.csv: 356 rows
2-data_20241115_120829_pre.csv: 421 rows
2-data_20241115_121237_post.csv: 551 rows
3-data_20241115_134123_pre.csv: 97 rows
3-data_20241115_134545_post.csv: 1037 rows
4-data_20241115_151409_pre.csv: 1426 rows
4-data_20241115_151829_post.csv: 291 rows
5-data_20241115_152227_pre.csv: 64 rows
5-data_20241115_152414_post.csv: 615 rows
6-data_20241115_161731_pre.csv: 259 rows
6-data_20241115_162026_post.csv: 875 rows
7-data_20241115_171711_pre.csv: 259 rows
7-data_20241115_172102_post.csv: 551 rows
8-data_20241115_175503_pre.csv: 745 rows
8-data_20241115_175911_post.csv: 842 rows


In [28]:
ASSETS_DIR = '../assets'

grid_points = np.load(os.path.join(ASSETS_DIR, "boob_grid_12_21.npy"))

# Remove points that are outside of the boob mesh.
# Because those points have been skipped during the points mapping, their value is (0, 0, 0)
origin = np.array([0, 0, 0])
points_to_skip = np.where((grid_points == origin).all(axis=1))[0]
points_of_interest = np.delete(grid_points, points_to_skip, axis=0)

# Drop the columns that represent points out of the breast model.
# dataframes_poi = [df.drop(columns=points_to_skip) for df in dataframes]
dataframes_poi = {key: df.drop(columns=points_to_skip) for key, df in dataframes.items()}

In [29]:
# Create the point cloud and initialize the scalars
boob_meshpoint_cloud = pv.PolyData(points_of_interest)


K = 5

# Load the boob 3D model from file
boob_mesh = pv.read( '../assets/boob.obj')
boob_mesh.translate(np.array([0, -0.035, 0]), inplace=True)

# Create a kd-tree for quick nearest-neighbor lookup.
kdtree = KDTree(points_of_interest)

# Find the K nearest point_cloud points for each points in the boob mesh and calculate their respective distances
boob_mesh['nearest_points'] = kdtree.query(boob_mesh.points, k=K)[1]
boob_mesh['nearest_points_dist'] = kdtree.query(boob_mesh.points, k=K)[0]

# Step 1: Calculate the reciprocal of the distances
reciprocal_distances = 1 / boob_mesh['nearest_points_dist']

# Step 2: Square the reciprocal distances
squared_reciprocal_distances = reciprocal_distances ** 1.2


# Step 3: Normalize the weights
normalized_weights = squared_reciprocal_distances / \
    np.sum(squared_reciprocal_distances, axis=1, keepdims=True)

boob_mesh['pressure'] = np.zeros(boob_mesh.points.shape[0])
boob_mesh['max_pressure'] = boob_mesh['pressure']


In [32]:
calibration_data =  {key: df.iloc[:20].median() for key, df in dataframes_poi.items()} 

In [33]:
def interpolate_grid_on_model(grid_scalar):
    return np.sum(grid_scalar[boob_mesh['nearest_points']] * normalized_weights, axis=1)

In [61]:
relative_pressure =  {key: dataframes_poi[key] - calibration_data[key] for key in csvs}

total_pressure = {key: df.mean() for key, df in relative_pressure.items()}

pre_total_pressure = [df for key, df in total_pressure.items() if key.endswith('_pre.csv')]
post_total_pressure = [df for key, df in total_pressure.items() if key.endswith('_post.csv')]


# sum(pre_total_pressure)

p3 = pv.Plotter(border=False, shape=(1, 2))

p3.subplot(0, 0)
p3.add_title("Pre", font_size=8)
p3.add_mesh(boob_mesh, copy_mesh=True, scalars=interpolate_grid_on_model(np.array(sum(pre_total_pressure))), cmap='cool', point_size=10)


p3.subplot(0, 1)
p3.add_title("Post", font_size=8)
p3.add_mesh(boob_mesh, copy_mesh=True, scalars=interpolate_grid_on_model(np.array(sum(post_total_pressure))),  cmap='cool', point_size=10)

p3.link_views()

p3.show()

print(len(pre_total_pressure))
print(len(post_total_pressure))




Widget(value='<iframe src="http://localhost:60295/index.html?ui=P_0x3105f34d0_12&reconnect=auto" class="pyvist…

8
8
